In [1]:
import pandas as pd
import pickle
from langchain_community.vectorstores import chroma as Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_community.document_loaders import DataFrameLoader
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document
import json
from Text_preprocessing import Text_preprocessing

In [ ]:
##FSD_1555
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/fsd_1555_0601_21_59_22TO23_59_59.pkl"
dateFrom = "2023-06-01 22:59:45+00:00" 
dateTo = "2023-06-01 23:59:59+00:00" #200 tweets labelled

# Loading pandas dataframe from picke file
with open(dataPath, 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
# df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

#Get data between thresholds
threshold_datetime_lower = pd.to_datetime(dateFrom)
threshold_datetime_upper = pd.to_datetime(dateTo)
df = df[df['date'] >= threshold_datetime_lower]
df = df[df['date'] <= threshold_datetime_upper]
df_new  = df.drop_duplicates(subset=["text"], keep=False)
#Covert date to string
df['date'] = df['date'].astype(str)
df

In [3]:
##FSD_1777
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/FSD1777_Oct23.json"
dateFrom = "2023-10-19T09:00:00+00:00" #2023-10-19T18:58:41Z for 200 tweets
dateTo = "2023-10-19T18:00:00+00:00"

"""Load relevant fields of flood tags api json response"""
def json_dataloader(dataPath = dataPath, dateFrom = dateFrom, dateTo = dateTo):
    # Load json and extract relevant records in pandas df
    with open(dataPath, 'r') as json_file:
        response_dict = json.load(json_file)

    # Convert to pandas df    
    pd.set_option('display.max_colwidth', None)
    df = pd.DataFrame(response_dict)
    df['date'] = pd.to_datetime(df['date'])
    df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

    #Get data between thresholds
    threshold_datetime_lower = pd.to_datetime(dateFrom)
    threshold_datetime_upper = pd.to_datetime(dateTo)
    df = df[df['date'] >= threshold_datetime_lower]
    df = df[df['date'] <= threshold_datetime_upper]

    #Remove duplicates
    df  = df.drop_duplicates(subset=["text"], keep=False)
    #Pre-process
    preprocess = Text_preprocessing(df)
    df = preprocess.preprocess()
    #Covert date to string
    df['date'] = df['date'].astype(str)
    return df

# Load the data from source
data = json_dataloader()


In [4]:
def bgeEmbeddings():
    model_name = "BAAI/bge-large-en-v1.5"
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
    model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return model

In [4]:
embeddings = bgeEmbeddings()

documents = []
loader = DataFrameLoader(data, page_content_column="text")
documents.extend(loader.load())
#Create a vector store
# db = Chroma.Chroma("Langchain collection",embeddings)
db = Chroma.Chroma.from_documents(documents,
                                  embeddings,
                                  collection_metadata={"hnsw:space": "cosine"})
if db._client.list_collections() != None:

  for collection in db._client.list_collections():
    ids = collection.get()['ids']
    print('REMOVE %s document(s) from %s collection' % (str(len(ids)), collection.name))
    if len(ids): collection.delete(ids)

db = Chroma.Chroma.from_documents(documents,
                                  embeddings,
                                  collection_metadata={"hnsw:space": "cosine"})

/home/mbhatti/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


REMOVE 555 document(s) from langchain collection


In [5]:
# Get all embeddings
len(db._collection.get()['ids'])

555

In [203]:
retriever = db.as_retriever(search_kwargs={'k': 5})

In [ ]:
query = """Which locations have been evacuated and which locations are getting evacutaion orders?"""
docs = retriever.get_relevant_documents(query=query)
print(docs)

In [ ]:
#Testing UAE embeddings
from scipy import spatial
input = "Which locations have recieved an evacuation order?"
query = embeddings.embed_query (input)
docs = retriever.get_relevant_documents(query=input)

doc_vecs = []
for doc in docs:
    doc_vecs.append(embeddings.embed_query (doc.page_content))



In [156]:
cosine_sim = []
for dv in doc_vecs:
    cosine_sim.append(spatial.distance.cosine(query, dv))

    # print(len(dv))

In [10]:
query = """Aberdeenshire"""
# [d[1] for d in db.similarity_search_with_score(query, k=300)]
outp =  db.similarity_search_with_score(query, k=300)

In [133]:
i = 0
for element in cosine_sim:
    if element < 0.58:
        print(i)
    i = i+1

32


In [157]:
print(cosine_sim)

[0.34229330651467094, 0.3523527454933534, 0.370738919434058, 0.3409977837027661, 0.3808665840456119, 0.37973994344030293, 0.37730797238417757, 0.35122532465563394, 0.35122532465563394, 0.3729195770924556, 0.36091296423524044, 0.3950091916938203, 0.3859390293303776, 0.36490902204834186, 0.39579932435002785, 0.3863409525292969, 0.35497307444649684, 0.40484377180522346, 0.39177352713349833, 0.380038328370378, 0.380038328370378, 0.3817885659419147, 0.40081366993277057, 0.40081366993277057, 0.40081366993277057, 0.39835584357576426, 0.4116979428866232, 0.4033720395728032, 0.4146771739478383, 0.3868499953945115, 0.39574133157486924, 0.38353882442642395, 0.42065178207880416, 0.41864865979655863, 0.41864865979655863, 0.3945974962308513, 0.39757451131286503, 0.39757451131286503, 0.4041586242397557, 0.36429657053695186, 0.4197360858733562, 0.4197360858733562, 0.4197360858733562, 0.4194068209661198, 0.42830807349480526, 0.42830807349480526, 0.3977146733165594, 0.3871695418771349, 0.381373273994193

In [25]:
#New embeddging model
from transformers import AutoModel, AutoTokenizer

input_texts = [
    "what is the capital of China?",
    "how to implement quick sort in python?",
    "Beijing",
    "sorting algorithms"
]

model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = outputs.last_hidden_state[:, 0]

In [21]:
e = embeddings.tolist()


4

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain_experimental.text_splitter import SemanticChunker
from typing import List


class MyEmbeddings:
    def __init__(self):
        self.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t).tolist() for t in texts]


embeddings = MyEmbeddings()

# splitter = SemanticChunker(embeddings)

In [5]:
# creating custom embeddings with non-default embedding model

from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        ##
        #New embeddging model
        from transformers import AutoModel, AutoTokenizer


        model_path = 'Alibaba-NLP/gte-large-en-v1.5'
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

        # Tokenize the input texts
        batch_dict = tokenizer(input, max_length=8192, padding=True, truncation=True, return_tensors='pt')

        outputs = model(**batch_dict)
        embeddings = outputs.last_hidden_state[:, 0]
                
        return embeddings.tolist()
        
        
        
custom_embeddings=MyEmbeddingFunction()


In [ ]:
# embeddings = model

documents = []
loader = DataFrameLoader(data, page_content_column="text")
documents.extend(loader.load())
print(documents)

# db = Chroma.Chroma.from_documents(documents,
#                                   custom_embeddings,
#                                   collection_metadata={"hnsw:space": "cosine"})
